Make sure you download the 2016-2017 Household LSMS survey data for Malawi from https://microdata.worldbank.org/index.php/catalog/lsms. It should be in `../data/input/LSMS/malawi_2016`

In [1]:
import pandas as pd
import numpy as np
import os

In [19]:
# everything in comments I have not converted to Python3 but they don't matter for predicting consumption
# you can play around with the stata files and add additional information to predict if you want
df = pd.read_stata('../data/input/LSMS/malawi_2016/IHS4 Consumption Aggregate.dta')
df['cons'] = df['rexpagg']/(365) *df['adulteq'])
df['cons'] = df['cons']*107.62/(116.28*166.12)
df.rename(columns={'hh_wgt': 'weight'}, inplace=True)
df = df[['case_id', 'cons', 'weight', 'urban']] # grab these columns

df_geo = pd.read_stata('../data/input/LSMS/malawi_2016/HouseholdGeovariables_stata11/HouseholdGeovariablesIHS4.dta')
df_cords = df_geo[['case_id', 'HHID', 'lat_modified', 'lon_modified']]
df_cords.rename(columns={'lat_modified': 'lat', 'lon_modified': 'lon'}, inplace=True)
# mwi13.hha <- read.dta('data/input/LSMS/MWI_2016_IHS-IV_v02_M_Stata/HH_MOD_A_FILT.dta')

df_hhf = pd.read_stata('../data/input/LSMS/malawi_2016/HH_MOD_F.dta')
# mwi13.room <- data.frame(hhid = mwi13.hhf$HHID, room = mwi13.hhf$hh_f10)
# mwi13.metal <- data.frame(hhid = mwi13.hhf$HHID, metal = mwi13.hhf$hh_f10=='IRON SHEETS')

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [22]:
df = pd.merge(df, df_cords[['case_id', 'HHID']], on='case_id')

In [23]:
df.head()

,case_id,cons,weight,urban,HHID
0,301025230225,815.565491,590.332886,RURAL,0001c970eecf473099368557e2080b3e
1,210374850204,4746.132812,265.032715,URBAN,000509f5cfcc4b078a09672b09425e95
2,311057710075,1281.518311,207.084106,RURAL,000bc107780044e59327dbf7ec960ac1
3,312048040073,1059.271973,207.147202,RURAL,000d1d26325d4f73a2ffbb8a99ab4752
4,311097790117,2277.215088,287.574097,RURAL,00104e33315844fdb2b8c6fdd35912a1


In [24]:
df_cords.head()

,case_id,HHID,lat,lon
0,301025230225,0001c970eecf473099368557e2080b3e,-14.683761,34.915074
1,210374850204,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591
2,311057710075,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503
3,312048040073,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219
4,311097790117,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629


In [25]:
df_combined = pd.merge(df, df_cords, on=['case_id', 'HHID'])

In [26]:
df_combined.head()

,case_id,cons,weight,urban,HHID,lat,lon
0,301025230225,815.565491,590.332886,RURAL,0001c970eecf473099368557e2080b3e,-14.683761,34.915074
1,210374850204,4746.132812,265.032715,URBAN,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591
2,311057710075,1281.518311,207.084106,RURAL,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503
3,312048040073,1059.271973,207.147202,RURAL,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219
4,311097790117,2277.215088,287.574097,RURAL,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629


In [27]:
df_combined.drop('case_id', axis=1, inplace=True)

In [28]:
df_combined.dropna(inplace=True) # can't use na values
df_combined.head()

,cons,weight,urban,HHID,lat,lon
0,815.565491,590.332886,RURAL,0001c970eecf473099368557e2080b3e,-14.683761,34.915074
1,4746.132812,265.032715,URBAN,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591
2,1281.518311,207.084106,RURAL,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503
3,1059.271973,207.147202,RURAL,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219
4,2277.215088,287.574097,RURAL,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629


In [29]:
df_combined.shape

(12444, 6)

In [30]:
clust_cons_avg = df_combined.groupby(['lat', 'lon']).mean().reset_index()[['lat', 'lon', 'cons']]

In [31]:
clust_cons_avg.head()

,lat,lon,cons
0,-17.095150,35.217213,1512.136108
1,-17.092351,35.114643,1192.031128
2,-17.016698,35.079629,1643.665283
3,-16.977243,35.205706,1481.665039
4,-16.956385,35.168967,894.125977


In [32]:
df_combined = pd.merge(df_combined.drop('cons', axis=1), clust_cons_avg, on=['lat', 'lon'])

In [33]:
df_combined.head()

,weight,urban,HHID,lat,lon,cons
0,590.332886,RURAL,0001c970eecf473099368557e2080b3e,-14.683761,34.915074,997.934204
1,590.332886,RURAL,2ca0d2ca3d714176963cc05e0c3f13e8,-14.683761,34.915074,997.934204
2,590.332886,RURAL,3c46ce7e9bfd4ef895f8c10685d63c62,-14.683761,34.915074,997.934204
3,590.332886,RURAL,3cdeb682067949cb973d17f6f5bf3221,-14.683761,34.915074,997.934204
4,590.332886,RURAL,52c072d210344f518716dd6822a20e6b,-14.683761,34.915074,997.934204


In [34]:
df_uniques = df_combined.drop_duplicates(subset=['lat', 'lon']); df_uniques.shape

(780, 6)

In [36]:
df_uniques[['lat', 'lon', 'cons']].to_csv('new.csv', index=False)

This shows how we can use the .tif file that we downloaded to get nightlight values.

In [1]:
import geoio
filename = '../data/input/Nightlights/2013/F182013.v4c_web.stable_lights.avg_vis.tif'
img = geoio.GeoImage(filename)
xPixel, yPixel = img.proj_to_raster(34.915074, -14.683761)

In [17]:
xPixel, yPixel

(25790.308983159237, 10762.551363048204)

In [18]:
im_array = np.squeeze(img.get_data())

In [19]:
im_array.shape

(16801, 43201)

In [20]:
im_array[int(yPixel),int(xPixel)] # this is how we grab the nightlight values!

0

In [21]:
import math

def create_space(lat, lon):
    # these are pulled from the paper to make the 10km^2 area
    return lat - (180/math.pi)*(5000/6378137), lon - (180/math.pi)*(5000/6378137)/math.cos(lat), \
            lat + (180/math.pi)*(5000/6378137), lon + (180/math.pi)*(5000/6378137)/math.cos(lat)

In [22]:
household_nightlights = []
for i,r in df_uniques.iterrows():
    min_lat, min_lon, max_lat, max_lon = create_space(r.lat, r.lon)
    xminPixel, yminPixel = img.proj_to_raster(min_lon, min_lat)
    xmaxPixel, ymaxPixel = img.proj_to_raster(max_lon, max_lat)
    
    xminPixel, xmaxPixel = min(xminPixel, xmaxPixel), max(xminPixel, xmaxPixel)
    yminPixel, ymaxPixel = min(yminPixel, ymaxPixel), max(yminPixel, ymaxPixel)
    
    xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
    household_nightlights.append(im_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())

In [23]:
df_uniques['nightlights'] = household_nightlights

/home/jupyter/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
os.makedirs('../data/output', exist_ok=True)
os.makedirs('../data/output/LSMS', exist_ok=True)
os.makedirs('../data/output/LSMS/malawi_2016/', exist_ok=True)

In [25]:
df_combined = pd.merge(df_combined, df_uniques[['lat', 'lon', 'nightlights']], on=['lat', 'lon'])

In [26]:
df_combined.head()

,weight,urban,HHID,lat,lon,cons,nightlights
0,590.332886,RURAL,0001c970eecf473099368557e2080b3e,-14.683761,34.915074,2.177948,0.0
1,590.332886,RURAL,2ca0d2ca3d714176963cc05e0c3f13e8,-14.683761,34.915074,2.177948,0.0
2,590.332886,RURAL,3c46ce7e9bfd4ef895f8c10685d63c62,-14.683761,34.915074,2.177948,0.0
3,590.332886,RURAL,3cdeb682067949cb973d17f6f5bf3221,-14.683761,34.915074,2.177948,0.0
4,590.332886,RURAL,52c072d210344f518716dd6822a20e6b,-14.683761,34.915074,2.177948,0.0


In [27]:
df_combined.to_csv('../data/output/LSMS/malawi_2016/Malawi-2016-LSMS-Household.csv', index=False)

In [28]:
df_combined.shape

(12444, 7)

In [29]:
# encode "RURAL" as 0 and "URBAN" as 1
df_combined['urban_encoded'] = pd.factorize(df_combined['urban'])[0]

In [30]:
clust_groups = df_combined.groupby(['lat', 'lon'])

In [31]:
clust_averages = clust_groups.mean().reset_index()

In [32]:
counts = clust_groups.count().reset_index()[['lat', 'lon', 'cons']]
counts.rename(columns={'cons': 'num_households'}, inplace=True)
clust_averages = pd.merge(clust_averages, counts, on=['lat', 'lon'])

In [33]:
# if more than 0.5 average within a clust, label it as 1 (URBAN), else 0
clust_averages['urban_encoded'] = clust_averages['urban_encoded'].apply(lambda x: round(x))

In [34]:
clust_averages['urban_encoded'] = clust_averages['urban_encoded'].apply(lambda x: 'RURAL' if x == 0 else 'URBAN')

In [35]:
clust_averages.rename(columns={'urban_encoded': 'urban'}, inplace=True)

In [36]:
clust_averages.head()

,lat,lon,weight,cons,nightlights,urban,num_households
0,-17.095150,35.217213,189.106400,2.039307,0.000000,RURAL,16
1,-17.092351,35.114643,165.955505,2.268112,0.000000,RURAL,16
2,-17.016698,35.079629,287.574097,2.336784,0.000000,RURAL,16
3,-16.977243,35.205706,223.751495,2.497626,0.121212,RURAL,16
4,-16.956385,35.168967,172.602493,1.688760,0.502674,RURAL,16


In [37]:
clust_averages.to_csv('../data/output/LSMS/malawi_2016/Malawi-2016-LSMS-Cluster.csv', index=False)

Create npy files

In [38]:
np.save('../data/output/LSMS/malawi_2016/lats.npy', clust_averages['lat'].values)
np.save('../data/output/LSMS/malawi_2016/lons.npy', clust_averages['lon'].values)
np.save('../data/output/LSMS/malawi_2016/consumptions.npy', clust_averages['cons'].values)
np.save('../data/output/LSMS/malawi_2016/nightlights.npy', clust_averages['nightlights'].values)
np.save('../data/output/LSMS/malawi_2016/households.npy', clust_averages['num_households'].values)